In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200916


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

ディレクトリ

In [5]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online\オンライン入会状況レポート")
data_dir2 = Path(r"C:\Users\crie072\Documents\21_Online")

In [31]:
#input
input_file = "【C136】新規入会者の利用状況調査（最新月：202007）"

input_file2="デジガレアフィリエイト申込累積"

code_list = "code-todoufuken"

#out_put
output_file = "新規入会属性"
output_file1 = "オンライン入会者の利用状況調査レポート作成用"
output_file2="オンライン入会者の利用状況調査レポート作成用_アフィリエイト別"

In [54]:
#CSCからの月次入会者リストを読込
df = pd.read_excel(data_dir /r"input"/f"{input_file}.xlsx",encoding='shift-jis'
                  ,dtyps={'入会申込番号':int})

In [55]:
df.head()

,実績月,入会申込番号,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,あとからリボ申込有無コード,Weｂ明細有無コード,解約,入会チャネル
0,202007,155051921528143,248,248,25,女性,福岡県,0,0,Ｄ_東京店,大丸松坂屋VISAカード,20200115,20200123,0,1,0,オンライン
1,202007,155051921528150,9,9,39,女性,大阪府,0,1,Ｄ_梅田店,さくらパンダVISAカード,20200203,20200213,0,1,0,オンライン
2,202007,155051921545014,395,395,58,男性,高知県,0,0,Ｄ_東京店,大丸松坂屋VISAゴールド,20200117,20200327,0,1,0,オンライン
3,202007,155051921551715,0,0,25,女性,大阪府,0,0,Ｄ_心斎橋店,大丸松坂屋VISAカード,20200220,0,0,1,1,オンライン
4,202007,155051921553752,0,0,63,女性,山口県,0,0,Ｄ_下関店,大丸松坂屋VISAカード,20200214,0,0,1,0,郵送


In [56]:
#重複削除
df_dup = df.drop_duplicates(subset=['入会申込番号','実績月'])
#df_dup = df.copy()
df_dup.shape[0]

33425

In [57]:
print(df.dtypes)

実績月               int64
入会申込番号            int64
直近６ケ月買物利用金額       int64
直近６ケ月全体利用金額       int64
年令                int64
性別               object
都道府県名            object
顧客状況コ－ド          object
マイペ設定             int64
DM希望店コード         object
カード券種名           object
カ－ド加入日            int64
初回利用日             int64
あとからリボ申込有無コード     int64
Weｂ明細有無コード        int64
解約                int64
入会チャネル           object
dtype: object


In [58]:
df_dup.head(2)

,実績月,入会申込番号,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,あとからリボ申込有無コード,Weｂ明細有無コード,解約,入会チャネル
0,202007,155051921528143,248,248,25,女性,福岡県,0,0,Ｄ_東京店,大丸松坂屋VISAカード,20200115,20200123,0,1,0,オンライン
1,202007,155051921528150,9,9,39,女性,大阪府,0,1,Ｄ_梅田店,さくらパンダVISAカード,20200203,20200213,0,1,0,オンライン


In [60]:
#都道府県コードリスト
code_list = pd.read_excel(data_dir /r"input"/"code-todoufuken.xlsx")
code_list.head()

,prefecture,prefecture2
0,北海道,01_北海道
1,青森県,02_青森県
2,岩手県,03_岩手県
3,宮城県,04_宮城県
4,秋田県,05_秋田県


In [61]:
_df_dup = df_dup.copy() #エラー回避
_df_dup['年代'] = _df_dup['年令']//10 * 10
df_dup = _df_dup.copy()

In [63]:
df_dup = df_dup.rename(columns={'実績月':'cp',
                        '入会申込番号':'applino',
                        '直近６ケ月全体利用金額':'credit_amt_6m',
                        '入会チャネル':'channel',
                        '解約':'kaiyaku_f',
                        '初回利用日':'first_use_dt',
                        'カ－ド加入日':'kanyu_dt',
                        'カード券種名':'card',
                        '性別':'gender',
                        '年代':'age',
                        '都道府県名':'prefecture',
                        'Weｂ明細有無コード':'web_f'})
df_dup = df_dup.drop(columns={'直近６ケ月買物利用金額','顧客状況コ－ド','マイペ設定','DM希望店コード','年令','あとからリボ申込有無コード'})
df_dup.head(2)

,cp,applino,credit_amt_6m,gender,prefecture,card,kanyu_dt,first_use_dt,web_f,kaiyaku_f,channel,age
0,202007,155051921528143,248,女性,福岡県,大丸松坂屋VISAカード,20200115,20200123,1,0,オンライン,20
1,202007,155051921528150,9,女性,大阪府,さくらパンダVISAカード,20200203,20200213,1,0,オンライン,30


In [64]:
df_dup = pd.merge(df_dup,code_list, on =['prefecture'], how = 'left')
df_dup.head()

,cp,applino,credit_amt_6m,gender,prefecture,card,kanyu_dt,first_use_dt,web_f,kaiyaku_f,channel,age,prefecture2
0,202007,155051921528143,248,女性,福岡県,大丸松坂屋VISAカード,20200115,20200123,1,0,オンライン,20,40_福岡県
1,202007,155051921528150,9,女性,大阪府,さくらパンダVISAカード,20200203,20200213,1,0,オンライン,30,27_大阪府
2,202007,155051921545014,395,男性,高知県,大丸松坂屋VISAゴールド,20200117,20200327,1,0,オンライン,50,39_高知県
3,202007,155051921551715,0,女性,大阪府,大丸松坂屋VISAカード,20200220,0,1,1,オンライン,20,27_大阪府
4,202007,155051921553752,0,女性,山口県,大丸松坂屋VISAカード,20200214,0,1,0,郵送,60,35_山口県


In [65]:
print(df_dup.dtypes)

cp                int64
applino           int64
credit_amt_6m     int64
gender           object
prefecture       object
card             object
kanyu_dt          int64
first_use_dt      int64
web_f             int64
kaiyaku_f         int64
channel          object
age               int64
prefecture2      object
dtype: object


In [66]:
df_dup['cp_dt'] = df_dup['cp']*100+1
df_dup['cp_dt'] = pd.to_datetime(df_dup['cp_dt'], format='%Y%m%d')

In [67]:
df_dup['kanyu_dt']=pd.to_datetime(df_dup['kanyu_dt'], format='%Y%m%d')
df_dup['kanyu_year'] = df_dup['kanyu_dt'].dt.year
df_dup['kanyu_month'] = df_dup['kanyu_dt'].dt.month

df_dup['kanyu_yymm'] = df_dup[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)

In [68]:
#経過月数
df_dup['mob']=(df_dup['cp_dt'].dt.year - df_dup['kanyu_dt'].dt.year) * 12 + df_dup['cp_dt'].dt.month - df_dup['kanyu_dt'].dt.month
df_dup.head(2)

,cp,applino,credit_amt_6m,gender,prefecture,card,kanyu_dt,first_use_dt,web_f,kaiyaku_f,channel,age,prefecture2,cp_dt,kanyu_year,kanyu_month,kanyu_yymm,mob
0,202007,155051921528143,248,女性,福岡県,大丸松坂屋VISAカード,2020-01-15,20200123,1,0,オンライン,20,40_福岡県,2020-07-01,2020,1,2020年1月,6
1,202007,155051921528150,9,女性,大阪府,さくらパンダVISAカード,2020-02-03,20200213,1,0,オンライン,30,27_大阪府,2020-07-01,2020,2,2020年2月,5


In [69]:
print(df_dup.dtypes)

cp                        int64
applino                   int64
credit_amt_6m             int64
gender                   object
prefecture               object
card                     object
kanyu_dt         datetime64[ns]
first_use_dt              int64
web_f                     int64
kaiyaku_f                 int64
channel                  object
age                       int64
prefecture2              object
cp_dt            datetime64[ns]
kanyu_year                int64
kanyu_month               int64
kanyu_yymm               object
mob                       int64
dtype: object


In [70]:
print(df_dup['card'].unique())

['大丸松坂屋VISAカード' 'さくらパンダVISAカード' '大丸松坂屋VISAゴールド' '大丸松坂屋マスターカード'
 '大丸松坂屋マスターゴールド' '大丸松坂屋VISAカード（学生）' '大丸松坂屋マスターカード（学生）' 'さくらパンダVISAカード（学生）'
 '大丸松坂屋お得意様ゴールド（マスター）' '大丸松坂屋お得意様ゴールド（VISA）' '大丸松坂屋お得意様VISA（下関）'
 '大丸松坂屋VISAカード（ヒューリック）' '大丸松坂屋マスターカード（大阪学校生協）' '大丸松坂屋従業員マスターカード'
 '大丸松坂屋従業員VISAカード' '大丸松坂屋お得意様VISA（高知）' '大丸松坂屋お得意様マスター（下関）'
 '大丸松坂屋お得意様マスター（高知）' '※大丸松坂屋VISAゴールド']


In [71]:
q = """
        SELECT
        cp
        ,channel
        ,kanyu_year
        ,kanyu_month
        ,kanyu_yymm
        ,age
        ,gender
        ,web_f
        ,prefecture2
        ,count(applino) as sum_no_account
        from 
            df_dup
        where 
            mob = 0
        group by
            1,2,3,4,5,6,7,8,9
        """
df_new = sqldf(q, locals())
df_new.head()

,cp,channel,kanyu_year,kanyu_month,kanyu_yymm,age,gender,web_f,prefecture2,sum_no_account
0,202007,オンライン,2020,7,2020年7月,10,女性,1,23_愛知県,1
1,202007,オンライン,2020,7,2020年7月,10,男性,1,25_滋賀県,1
2,202007,オンライン,2020,7,2020年7月,10,男性,1,26_京都府,1
3,202007,オンライン,2020,7,2020年7月,20,女性,1,01_北海道,5
4,202007,オンライン,2020,7,2020年7月,20,女性,1,04_宮城県,2


In [85]:
#CSV出力（新規属性）
df_new.to_csv(data_dir /r"Output"/f"{output_file}_{input_file}.csv", index=None,header=True,encoding='shift-jis')

In [72]:
q = """
        SELECT
        cp
        ,channel
        ,case
            when card like '%お得意様%' then '外商'
            when card like '%ゴールド%' then 'ゴールド'
            else '一般'
        end as card_cate
        ,kanyu_year
        ,kanyu_month
        ,kanyu_yymm
        ,mob
        ,count(applino) as sum_no_account
        ,sum(case when first_use_dt > 0 then 1 else 0 end) as sum_no_1st_use
        ,sum(case when kaiyaku_f = 1 then 1 else 0 end) as sum_no_kaiyaku
        ,sum(credit_amt_6m) as sum_credit_amt6m
        from 
            df_dup
        group by
            1,2,3,4,5,6,7
        """
df_sql = sqldf(q, locals())
df_sql.head()

,cp,channel,card_cate,kanyu_year,kanyu_month,kanyu_yymm,mob,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m
0,202007,オンライン,ゴールド,2020,1,2020年1月,6,29,22,0,11572
1,202007,オンライン,ゴールド,2020,2,2020年2月,5,21,17,1,10504
2,202007,オンライン,ゴールド,2020,3,2020年3月,4,23,17,1,4650
3,202007,オンライン,ゴールド,2020,4,2020年4月,3,21,17,0,10462
4,202007,オンライン,ゴールド,2020,5,2020年5月,2,20,11,0,4693


In [73]:
#初回利用率
df_sql['rate_1st_use'] = df_sql['sum_no_1st_use']/df_sql['sum_no_account']

#クレジット初回利用単価
df_sql['unit_1st_amt'] = df_sql['sum_credit_amt6m']/df_sql['sum_no_1st_use']

#解約率
df_sql['rate_kaiyaku'] = df_sql['sum_no_kaiyaku']/df_sql['sum_no_account']


df_sql.head()

,cp,channel,card_cate,kanyu_year,kanyu_month,kanyu_yymm,mob,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m,rate_1st_use,unit_1st_amt,rate_kaiyaku
0,202007,オンライン,ゴールド,2020,1,2020年1月,6,29,22,0,11572,0.758621,526.000000,0.000000
1,202007,オンライン,ゴールド,2020,2,2020年2月,5,21,17,1,10504,0.809524,617.882353,0.047619
2,202007,オンライン,ゴールド,2020,3,2020年3月,4,23,17,1,4650,0.739130,273.529412,0.043478
3,202007,オンライン,ゴールド,2020,4,2020年4月,3,21,17,0,10462,0.809524,615.411765,0.000000
4,202007,オンライン,ゴールド,2020,5,2020年5月,2,20,11,0,4693,0.550000,426.636364,0.000000


CSVアウトプット

In [74]:
df_sql.to_csv(data_dir /r"Output"/f"{output_file1}_{input_file}.csv", index=None,header=True,encoding='utf-8-sig')

アフィリエイト別<br>積み残しメモ<br>アフィリエイトの累積承認リストを正しく作る必要あり

In [75]:
print(input_file2)

デジガレアフィリエイト申込累積


In [76]:
bk = pd.ExcelFile(data_dir2/r"input"/f"{input_file2}.xlsx")
print(bk.sheet_names)

['統合（通常）', '統合（利用条件付き）']


In [77]:
df1 = pd.read_excel(data_dir2 /r"input"/f"{input_file2}.xlsx", sheet_name=0)
df2 = pd.read_excel(data_dir2 /r"input"/f"{input_file2}.xlsx", sheet_name=1)
df_affi = pd.concat([df1,df2])
df_affi

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価
0,37.0,げん玉,ポイント,2020-01-08 18:43:29,2020-01-08 18:56:06,155051921643363,BGT,9000
1,37.0,げん玉,ポイント,2020-01-08 19:11:08,2020-01-08 19:32:05,155051921643397,BGT,9000
2,82.0,ちょびリッチ。ドットコム,ポイント,2020-01-08 20:07:13,2020-01-08 20:18:06,155051921643538,BGT,9000
3,52.0,ECナビSP,ポイント,2020-01-08 20:21:44,2020-01-08 20:31:05,155051921643553,BGT,9000
4,52.0,ECナビSP,ポイント,2020-01-08 20:31:50,2020-01-08 20:39:06,155051921643587,BGT,9000
...,...,...,...,...,...,...,...,...
2523,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-18 04:07:00,2020-06-18 04:15:11,155051921922890,BGT,9000
2524,57.0,ライフメディア,ポイント,2020-06-18 18:44:50,2020-06-18 19:03:12,155051921923690,BGT,9000
2525,37.0,げん玉,ポイント,2020-06-18 20:53:46,2020-06-18 21:10:13,155051921923930,BGT,9000
2526,57.0,ライフメディア,ポイント,2020-06-19 06:45:51,2020-06-19 07:13:12,155051921924177,BGT,9000


In [78]:
df_affi = df_affi.rename(columns={'メディア名':'media',
                                  '媒体チャネル':'baitai',
                                  '参照用':'applino'})
df_affi = df_affi[['media','baitai','applino']]
df_affi.head(2)

,media,baitai,applino
0,げん玉,ポイント,155051921643363
1,げん玉,ポイント,155051921643397


In [79]:
q = """
        SELECT
        cp
        ,channel
        ,kanyu_year
        ,kanyu_month
        ,kanyu_yymm
        ,mob
        ,media
        ,baitai
        ,count(t1.applino) as sum_no_account
        ,sum(case when first_use_dt > 0 then 1 else 0 end) as sum_no_1st_use
        ,sum(case when kaiyaku_f = 1 then 1 else 0 end) as sum_no_kaiyaku
        ,sum(credit_amt_6m) as sum_credit_amt6m
        from 
            df_dup t1
        inner join
            df_affi t2
        on t1.applino = t2.applino
        group by
            1,2,3,4,5,6,7,8
        """

In [80]:
df_sql = sqldf(q, locals())

In [81]:
#初回利用率
df_sql['rate_1st_use'] = df_sql['sum_no_1st_use']/df_sql['sum_no_account']

#クレジット初回利用単価
df_sql['unit_1st_amt'] = df_sql['sum_credit_amt6m']/df_sql['sum_no_1st_use']

#解約率
df_sql['rate_kaiyaku'] = df_sql['sum_no_kaiyaku']/df_sql['sum_no_account']


df_sql.head()

,cp,channel,kanyu_year,kanyu_month,kanyu_yymm,mob,media,baitai,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m,rate_1st_use,unit_1st_amt,rate_kaiyaku
0,202007,オンライン,2020,1,2020年1月,6,CLUBPanasonicコイン,ポイント,4,4,0,105,1.000000,26.25,0.000000
1,202007,オンライン,2020,1,2020年1月,6,ECナビPC,ポイント,7,2,2,16,0.285714,8.00,0.285714
2,202007,オンライン,2020,1,2020年1月,6,ECナビSP,ポイント,9,4,3,25,0.444444,6.25,0.333333
3,202007,オンライン,2020,1,2020年1月,6,GetMoney!(JIPC参加企業)（連携用）,ポイント,12,2,3,25,0.166667,12.50,0.250000
4,202007,オンライン,2020,1,2020年1月,6,NetMile（すぐたま）,ポイント,12,6,4,186,0.500000,31.00,0.333333


In [82]:
df_sql.head()

,cp,channel,kanyu_year,kanyu_month,kanyu_yymm,mob,media,baitai,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m,rate_1st_use,unit_1st_amt,rate_kaiyaku
0,202007,オンライン,2020,1,2020年1月,6,CLUBPanasonicコイン,ポイント,4,4,0,105,1.000000,26.25,0.000000
1,202007,オンライン,2020,1,2020年1月,6,ECナビPC,ポイント,7,2,2,16,0.285714,8.00,0.285714
2,202007,オンライン,2020,1,2020年1月,6,ECナビSP,ポイント,9,4,3,25,0.444444,6.25,0.333333
3,202007,オンライン,2020,1,2020年1月,6,GetMoney!(JIPC参加企業)（連携用）,ポイント,12,2,3,25,0.166667,12.50,0.250000
4,202007,オンライン,2020,1,2020年1月,6,NetMile（すぐたま）,ポイント,12,6,4,186,0.500000,31.00,0.333333


In [83]:
df_sql.shape[0]

194

In [84]:
df_sql.to_csv(data_dir /r"Output"/f"{output_file2}_{input_file}.csv", index=None,header=True,encoding='utf-8-sig')